In [26]:
class table_order():
    def __init__(self) -> None:
        self.prepare()
        self.create_orders()

    def prepare(self):
        from A01_TOOL_BOX import tool_box
        tool_box=tool_box()

        self.pandas=tool_box.pandas
        self.random=tool_box.random
        self.datetime=tool_box.datetime

        self.sales_orders=self.pandas.read_csv('NEW_DATASET/A03_TAB_CLIENTS.csv',sep=';',decimal=',')[['ID_client','cluster']]

    def create_orders(self):
        display(self.sales_orders)

table_order()

,ID_client,cluster
0,10001-001,C
1,10001-002,D
2,10001-003,C
3,10001-004,D
4,10001-005,A
...,...,...
538,98101-039,D
539,98101-040,C
540,98101-041,D
541,98101-042,D
